### Script to plot regridded MPAS fields comparing the original and revised PV diagnostics package versions ###

#### *Figs 3-6 in JAMES paper*
---

Manda Chasteen<br>
April 2024

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# My colormaps file:
import custom_colormaps as cc

np.set_printoptions(suppress=True)

import warnings
warnings.simplefilter("ignore") 

In [ ]:
parent_dir = ''     # Add file directory here 

file_old = xr.open_dataset(parent_dir+'pvbudget15to3km_single_original_20190520_03:00:36.nc', chunks={'longitude' : 100, 'latitude' : 100, 'nVertLevels' : 5}).squeeze()
file_new = xr.open_dataset(parent_dir+'pvbudget15to3km_single_fixed_20190520_03:00:36.nc', chunks={'longitude' : 100, 'latitude' : 100, 'nVertLevels' : 5}).squeeze()

In [ ]:
# Set time of files 
t_end = pd.Timestamp(2019, 5, 20, 3, 0, 36) 

#### Plot PV fields (multiplied by sign of latitude) and PV difference (Fig. 3) ####

In [ ]:
# Specify vertical level to plot:
lev = 31
level = str(lev+1)

fold = file_old.sel(nVertLevels=lev).squeeze()
fnew = file_new.sel(nVertLevels=lev).squeeze()

In [ ]:
pv_old = fold.ertel_pv
pv_new = fnew.ertel_pv
pv_diff = pv_new - pv_old

lat_sign = np.sign(fold.latitude)

In [ ]:
print('Minimum PV difference: ', pv_diff.min().values)
print('Maximum PV difference: ', pv_diff.max().values)

In [ ]:
### To plot pcolormesh with gouraud on Robinson proj not centered at 0.0, we need to restructure arrays around desired central longitude
### Want central longitude to be -95.0 degrees, so rightmost plot bounds will become -95 + 180.0, and points beyond that will be transposed 
### onto left side of plot. We first do this to longitude array and then arrays we are plotting.
### Need to do the same thing with the arrays 

lon_central = -95.0

# Restructure longitude array:
lons_transposed = file_old.longitude[fold.longitude > lon_central+180.0]
lons_keep = file_old.longitude[fold.longitude <= lon_central+180.0]
lons_new = xr.concat([lons_transposed, lons_keep], dim='longitude')

# Restructure plotting arrays:
pv_old_transposed = pv_old.sel(longitude=lons_transposed)
pv_old_keep = pv_old.sel(longitude=lons_keep)
pv_old_new = xr.concat([pv_old_transposed, pv_old_keep], dim='longitude')

pv_new_transposed = pv_new.sel(longitude=lons_transposed)
pv_new_keep = pv_new.sel(longitude=lons_keep)
pv_new_new = xr.concat([pv_new_transposed, pv_new_keep], dim='longitude')

In [ ]:
### Make 3-panel plot of PV and PV difference:

plt.rcParams['savefig.dpi'] = 250
proj = ccrs.Robinson(central_longitude=lon_central, globe=None)
fig, ax = plt.subplots(3,1,figsize=(20,35),subplot_kw={'projection': proj}) 
fig.patch.set_facecolor('white')
fig.patch.set_alpha(1)
ax = ax.flatten()

#######
# Set up map kwargs:
bcolor = 'black' 
lwidth_border = 0.8

for i in np.arange(0,np.shape(ax)[0],1):
    ax[i].coastlines(resolution='50m', color=bcolor, linewidth=lwidth_border, zorder=2, alpha=1)
    ax[i].set_rasterization_zorder(1)

# Explicitly set PV contour levels and norm -- works with cc.manda_pv() colormap
bounds = np.array([-4, -3.5, -3, -2.5, -2, -1.5, -1, -0.5, -0.25, 0, 0.25, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6, 7, 8, 9, 10, 12, 15, 20])
norm = colors.BoundaryNorm(boundaries=bounds, ncolors=120, extend='both')

#######
# Specify plots:

# PV x sign(f) -- original package:
pv_plot1 = ax[0].pcolormesh(lons_new, fold.latitude, pv_old_new*lat_sign, norm=norm, shading='gouraud', cmap=cc.manda_pv(), transform=ccrs.PlateCarree(), zorder=0) 
cb = fig.colorbar(pv_plot1, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[0], ticks=bounds)
cb.ax.tick_params(labelsize=14, labelrotation=-40.0) 

ax[0].set_title(r"Ertel's PV $\times\ sign(f)$ $-$ Original Package", fontsize=21.5, y=1.02, fontweight='medium', loc='left', horizontalalignment='left')
ax[0].set_title(str(t_end.day).zfill(2)+' '+str(t_end.month_name())+' '+str(t_end.year)+'\n '+r' $t =$ '+str(t_end)[-8:], 
                fontsize=12.5,y=0.89, linespacing=1.5, fontweight='medium', loc='right', color='gray', zorder=12)
ax[0].text(1.0, 1.06, 'Model Level '+level, fontsize=12.5, fontweight='medium', va='top', ha='right', style='italic', transform=ax[0].transAxes, color='dimgray', zorder=12) 


# PV x sign(f) -- revised package:
pv_plot2 = ax[1].pcolormesh(lons_new, fold.latitude, pv_new_new*lat_sign, norm=norm, shading='gouraud', cmap=cc.manda_pv(), transform=ccrs.PlateCarree(), zorder=0) 
cb2 = fig.colorbar(pv_plot2, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[1], ticks=bounds)
cb2.ax.tick_params(labelsize=14, labelrotation=-40.0)

ax[1].set_title(r"Ertel's PV $\times\ sign(f)$ $-$ Revised Package", fontsize=21.5, y=1.02, fontweight='medium', loc='left', horizontalalignment='left')


# PV difference:
diff_plot = ax[2].pcolormesh(lons_new, file_old.latitude, (pv_new_new-pv_old_new), vmin=-0.5, vmax=0.5, cmap=cc.adobe_rd_bu(),shading='gouraud', transform=ccrs.PlateCarree(), zorder=0)  
cb2 = fig.colorbar(diff_plot, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[2], ticks=np.arange(-0.5,0.6,0.1))
cb2.ax.tick_params(labelsize=14) 

ax[2].set_title(r'Difference [Revised - Original]', fontsize=21.5, y=1.02, fontweight='medium', loc='left', horizontalalignment='left')


figname = 'ertelpv_old_new_level'+level+'_03.00.36.pdf'
#fig.savefig(figname,bbox_inches="tight")
print('saved')

plt.show()

#### Plot instantaneous diabatic and frictional tendencies for original and revised versions (Figs. 4 and 5) ####

In [ ]:
# Specify vertical levels to plot:
lev = [9, 31]
levels = [str(x+1) for x in lev]

fold = file_old.sel(nVertLevels=lev).squeeze()
fnew = file_new.sel(nVertLevels=lev).squeeze()

In [ ]:
# Either extract model time step explicltly from file (not included in regridded file attributes by default, so need to obtain it from native file) 
# or set to known value

timestep_dir = ''     # Add directory with native files here 
dt = xr.open_dataset(timestep_dir+'pvbudget15to3.2019-05-20_03.00.36_ddzold.nc').attrs['config_dt']
# dt = 18.0

tdelt = pd.Timedelta(dt, 's')
t_start = t_end - tdelt

In [ ]:
# Tendencies for a given time step are written out at the end of the timestep, so for the timestep from 03:00:18Z - 03:00:36Z, we only need 
# the latter file. 

diab_old = fold.depv_dt_diab
fric_old = fold.depv_dt_fric

diab_new = fnew.depv_dt_diab
fric_new = fnew.depv_dt_fric

In [ ]:
### To plot pcolormesh with gouraud on Robinson proj not centered at 0.0, we need to restructure arrays around desired central longitude
### Want central longitude to be -95.0 degrees, so rightmost plot bounds will become -95 + 180.0, and points beyond that will be transposed 
### onto left side of plot. We first do this to longitude array and then arrays we are plotting.
### Need to do the same thing with the arrays 

lon_central = -95.0

# Restructure longitude array:
lons_transposed = file_old.longitude[fold.longitude > lon_central+180.0]
lons_keep = file_old.longitude[fold.longitude <= lon_central+180.0]
lons_new = xr.concat([lons_transposed, lons_keep], dim='longitude')

# Restructure plotting arrays:
diab_old_transposed = diab_old.sel(longitude=lons_transposed)
diab_old_keep = diab_old.sel(longitude=lons_keep)
diab_old_new = xr.concat([diab_old_transposed, diab_old_keep], dim='longitude')

diab_new_transposed = diab_new.sel(longitude=lons_transposed)
diab_new_keep = diab_new.sel(longitude=lons_keep)
diab_new_new = xr.concat([diab_new_transposed, diab_new_keep], dim='longitude')

fric_old_transposed = fric_old.sel(longitude=lons_transposed)
fric_old_keep = fric_old.sel(longitude=lons_keep)
fric_old_new = xr.concat([fric_old_transposed, fric_old_keep], dim='longitude')

fric_new_transposed = fric_new.sel(longitude=lons_transposed)
fric_new_keep = fric_new.sel(longitude=lons_keep)
fric_new_new = xr.concat([fric_new_transposed, fric_new_keep], dim='longitude')

##### Plot diabatic tendency:

In [ ]:
# Makes 3 panel plot of diabatic tendency for a given vertical level. 

counter = -1
for lev_ind in lev:
    counter += 1

    plt.rcParams['savefig.dpi'] = 250
    proj = ccrs.Robinson(central_longitude=lon_central, globe=None)
    fig, ax = plt.subplots(3,1,figsize=(20,35),subplot_kw={'projection': proj}) 
    fig.patch.set_facecolor('white')
    fig.patch.set_alpha(1)
    ax = ax.flatten()
    
    #######
    # Set up map kwargs:
    bcolor = 'black' 
    lwidth_border = 0.8
    
    for i in np.arange(0,np.shape(ax)[0],1):
        ax[i].coastlines(resolution='50m', color=bcolor, linewidth=lwidth_border, zorder=2, alpha=1)
        ax[i].set_rasterization_zorder(1)
    
    #######
    # Specify plots:

    # Diabatic PV tendency - original version:
    if counter == 0:
        tend1 = ax[0].pcolormesh(lons_new, file_old.latitude, diab_old_new[:,:,counter]*1e5, vmin=-10, vmax=10, cmap=cc.seaborn_mako_rocket_strong(),shading='gouraud',transform=ccrs.PlateCarree(),zorder=0)  
        cb = fig.colorbar(tend1, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[0], ticks=np.arange(-10,11,1))
        cb.ax.tick_params(labelsize=14) 
        ax[0].text(0.87, -0.05, r'[$\times\ 10^{-5}$ PVU s$^{-1}$]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[0].transAxes, color='gray', zorder=12) 
    else:
        tend1 = ax[0].pcolormesh(lons_new, file_old.latitude, diab_old_new[:,:,counter]*1e6, vmin=-10, vmax=10, cmap=cc.seaborn_mako_rocket_strong(),shading='gouraud',transform=ccrs.PlateCarree(),zorder=0)  
        cb = fig.colorbar(tend1, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[0], ticks=np.arange(-10,11,1))
        cb.ax.tick_params(labelsize=14) 
        ax[0].text(0.87, -0.05, r'[$\times\ 10^{-6}$ PVU s$^{-1}$]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[0].transAxes, color='gray', zorder=12) 

    ax[0].set_title(r'Diabatic PV Tendency Over $\Delta t$ $-$ Original Package',fontsize=21.5,y=1.02,fontweight='medium', loc='left', horizontalalignment='left')
    ax[0].set_title(str(t_start.day).zfill(2)+' '+str(t_start.month_name())+' '+str(t_start.year)+'\n '+r' $t_1 =$ '+str(t_start)[-8:]+'\n '+r'  $t_2 =$ '+str(t_end)[-8:], 
                    fontsize=15.5,y=0.89, linespacing=1.5, fontweight='medium', loc='right', color='gray', zorder=12)
    ax[0].text(1.0, 1.06, 'Model Level '+levels[counter], fontsize=17.5, fontweight='medium', va='top', ha='right', style='italic', transform=ax[0].transAxes, color='dimgray', zorder=12) 


    # Diabatic PV tendency - revised version:
    if counter == 0:
        tend2 = ax[1].pcolormesh(lons_new, file_old.latitude, diab_new_new[:,:,counter]*1e5, vmin=-10, vmax=10, cmap=cc.seaborn_mako_rocket_strong(),shading='gouraud',transform=ccrs.PlateCarree(),zorder=0)  
        cb1 = fig.colorbar(tend1, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[1], ticks=np.arange(-10,11,1))
        cb1.ax.tick_params(labelsize=14) 
        ax[1].text(0.87, -0.05, r'[$\times\ 10^{-5}$ PVU s$^{-1}$]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[1].transAxes, color='gray', zorder=12) 
    else:
        tend2 = ax[1].pcolormesh(lons_new, file_old.latitude, diab_new_new[:,:,counter]*1e6, vmin=-10, vmax=10, cmap=cc.seaborn_mako_rocket_strong(),shading='gouraud',transform=ccrs.PlateCarree(),zorder=0)  
        cb1 = fig.colorbar(tend1, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[1], ticks=np.arange(-10,11,1))
        cb1.ax.tick_params(labelsize=14) 
        ax[1].text(0.87, -0.05, r'[$\times\ 10^{-6}$ PVU s$^{-1}$]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[1].transAxes, color='gray', zorder=12) 

    ax[1].set_title(r'Diabatic PV Tendency Over $\Delta t$ $-$ Revised Package', fontsize=21.5, y=1.02, fontweight='medium', loc='left', horizontalalignment='left')

    
    # Difference:
    if counter == 0:
        diff_plot = ax[2].pcolormesh(lons_new, file_old.latitude, (diab_new_new[:,:,counter]-diab_old_new[:,:,counter])*1e5, vmin=-10, vmax=10, cmap=cc.adobe_rd_bu(),shading='gouraud',transform=ccrs.PlateCarree(),zorder=0)  
        cb2 = fig.colorbar(diff_plot, extend='both',orientation="horizontal",pad=0.03,fraction=0.037,shrink=0.97,aspect=35,ax=ax[2], ticks=np.arange(-10,11,1))
        cb2.ax.tick_params(labelsize=14) 
        ax[2].text(0.87, -0.05, r'[$\times\ 10^{-5}$ PVU s$^{-1}$]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[2].transAxes, color='gray', zorder=12) 
    else:
        diff_plot = ax[2].pcolormesh(lons_new, file_old.latitude, (diab_new_new[:,:,counter]-diab_old_new[:,:,counter])*1e6, vmin=-10, vmax=10, cmap=cc.adobe_rd_bu(),shading='gouraud',transform=ccrs.PlateCarree(),zorder=0)  
        cb2 = fig.colorbar(diff_plot, extend='both',orientation="horizontal",pad=0.03,fraction=0.037,shrink=0.97,aspect=35,ax=ax[2], ticks=np.arange(-10,11,1))
        cb2.ax.tick_params(labelsize=14) 
        ax[2].text(0.87, -0.05, r'[$\times\ 10^{-6}$ PVU s$^{-1}$]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[2].transAxes, color='gray', zorder=12) 

    ax[2].set_title(r'Difference [Revised - Original]',fontsize=21.5, y=1.02, fontweight='medium', loc='left', horizontalalignment='left')


    figname = 'pv_diabtend_old_new_level'+levels[counter]+'_03.00.18.pdf'
    #fig.savefig('figname,bbox_inches="tight")
    print('saved')
    plt.show()


In [ ]:
# Makes 3 panel plot of frictional tendency for a given vertical level. 

counter = -1
for lev_ind in lev:
    counter += 1

    plt.rcParams['savefig.dpi'] = 250
    proj = ccrs.Robinson(central_longitude=lon_central, globe=None)
    fig, ax = plt.subplots(3,1,figsize=(20,35),subplot_kw={'projection': proj}) 
    fig.patch.set_facecolor('white')
    fig.patch.set_alpha(1)
    ax = ax.flatten()
    
    #######
    # Set up map kwargs:
    bcolor = 'black' 
    lwidth_border = 0.8
    
    for i in np.arange(0,np.shape(ax)[0],1):
        ax[i].coastlines(resolution='50m', color=bcolor, linewidth=lwidth_border, zorder=2, alpha=1)
        ax[i].set_rasterization_zorder(1)
    
    #######
    # Specify plots:

    # Frictional PV tendency - original version:
    if counter == 0:
        tend1 = ax[0].pcolormesh(lons_new, file_old.latitude, fric_old_new[:,:,counter]*1e5, vmin=-10, vmax=10, cmap=cc.seaborn_mako_rocket_strong(),shading='gouraud',transform=ccrs.PlateCarree(),zorder=0)  
        cb = fig.colorbar(tend1, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[0], ticks=np.arange(-10,11,1))
        cb.ax.tick_params(labelsize=14) 
        ax[0].text(0.87, -0.05, r'[$\times\ 10^{-5}$ PVU s$^{-1}$]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[0].transAxes, color='gray', zorder=12) 
    else:
        tend1 = ax[0].pcolormesh(lons_new, file_old.latitude, fric_old_new[:,:,counter]*1e6, vmin=-10, vmax=10, cmap=cc.seaborn_mako_rocket_strong(),shading='gouraud',transform=ccrs.PlateCarree(),zorder=0)  
        cb = fig.colorbar(tend1, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[0], ticks=np.arange(-10,11,1))
        cb.ax.tick_params(labelsize=14) 
        ax[0].text(0.87, -0.05, r'[$\times\ 10^{-6}$ PVU s$^{-1}$]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[0].transAxes, color='gray', zorder=12) 

    ax[0].set_title(r'Frictional PV Tendency Over $\Delta t$ $-$ Original Package',fontsize=21.5,y=1.02,fontweight='medium', loc='left', horizontalalignment='left')
    ax[0].set_title(str(t_start.day).zfill(2)+' '+str(t_start.month_name())+' '+str(t_start.year)+'\n '+r' $t_1 =$ '+str(t_start)[-8:]+'\n '+r'  $t_2 =$ '+str(t_end)[-8:], 
                    fontsize=15.5,y=0.89, linespacing=1.5, fontweight='medium', loc='right', color='gray', zorder=12)
    ax[0].text(1.0, 1.06, 'Model Level '+levels[counter], fontsize=17.5, fontweight='medium', va='top', ha='right', style='italic', transform=ax[0].transAxes, color='dimgray', zorder=12) 


    # Frictional PV tendency - revised version:
    if counter == 0:
        tend2 = ax[1].pcolormesh(lons_new, file_old.latitude, fric_new_new[:,:,counter]*1e5, vmin=-10, vmax=10, cmap=cc.seaborn_mako_rocket_strong(),shading='gouraud',transform=ccrs.PlateCarree(),zorder=0)  
        cb1 = fig.colorbar(tend1, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[1], ticks=np.arange(-10,11,1))
        cb1.ax.tick_params(labelsize=14) 
        ax[1].text(0.87, -0.05, r'[$\times\ 10^{-5}$ PVU s$^{-1}$]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[1].transAxes, color='gray', zorder=12) 
    else:
        tend2 = ax[1].pcolormesh(lons_new, file_old.latitude, fric_new_new[:,:,counter]*1e6, vmin=-10, vmax=10, cmap=cc.seaborn_mako_rocket_strong(),shading='gouraud',transform=ccrs.PlateCarree(),zorder=0)  
        cb1 = fig.colorbar(tend1, extend='both', orientation="horizontal", pad=0.03, fraction=0.037, shrink=0.97, aspect=35, ax=ax[1], ticks=np.arange(-10,11,1))
        cb1.ax.tick_params(labelsize=14) 
        ax[1].text(0.87, -0.05, r'[$\times\ 10^{-6}$ PVU s$^{-1}$]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[1].transAxes, color='gray', zorder=12) 

    ax[1].set_title(r'Frictional PV Tendency Over $\Delta t$ $-$ Revised Package', fontsize=21.5, y=1.02, fontweight='medium', loc='left', horizontalalignment='left')

    
    # Difference:
    if counter == 0:
        diff_plot = ax[2].pcolormesh(lons_new, file_old.latitude, (fric_new_new[:,:,counter]-fric_old_new[:,:,counter])*1e5, vmin=-10, vmax=10, cmap=cc.adobe_rd_bu(),shading='gouraud',transform=ccrs.PlateCarree(),zorder=0)  
        cb2 = fig.colorbar(diff_plot, extend='both',orientation="horizontal",pad=0.03,fraction=0.037,shrink=0.97,aspect=35,ax=ax[2], ticks=np.arange(-10,11,1))
        cb2.ax.tick_params(labelsize=14) 
        ax[2].text(0.87, -0.05, r'[$\times\ 10^{-5}$ PVU s$^{-1}$]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[2].transAxes, color='gray', zorder=12) 
    else:
        diff_plot = ax[2].pcolormesh(lons_new, file_old.latitude, (fric_new_new[:,:,counter]-fric_old_new[:,:,counter])*1e6, vmin=-10, vmax=10, cmap=cc.adobe_rd_bu(),shading='gouraud',transform=ccrs.PlateCarree(),zorder=0)  
        cb2 = fig.colorbar(diff_plot, extend='both',orientation="horizontal",pad=0.03,fraction=0.037,shrink=0.97,aspect=35,ax=ax[2], ticks=np.arange(-10,11,1))
        cb2.ax.tick_params(labelsize=14) 
        ax[2].text(0.87, -0.05, r'[$\times\ 10^{-6}$ PVU s$^{-1}$]', fontsize=15.5, fontweight='medium', ha='left', va='center', transform=ax[2].transAxes, color='gray', zorder=12) 

    ax[2].set_title(r'Difference [Revised - Original]',fontsize=21.5, y=1.02, fontweight='medium', loc='left', horizontalalignment='left')


    figname = 'pv_frictend_old_new_level'+levels[counter]+'_03.00.18.pdf'
    #fig.savefig('figname,bbox_inches="tight")
    print('saved')
    plt.show()


#### Plot dynamic tropopause (Fig. 6) ####

In [ ]:
# Original package:
ilev_dt_old = file_old.iLev_DT
theta_pv_old = file_old.theta_pv

# Revised package:
ilev_dt_new = file_new.iLev_DT
theta_pv_new = file_new.theta_pv

In [ ]:
### Make 4-panel plot of DT model level and potential temperature:
plt.rcParams['savefig.dpi'] = 250
proj = ccrs.Orthographic(central_longitude=-90.0, central_latitude=30.0) 
fig, axs = plt.subplots(2,2,figsize=(20,22),subplot_kw={'projection': proj}) 
fig.patch.set_facecolor('white')
fig.patch.set_alpha(1)
ax = axs.flatten()

#######
# Set up map kwargs:

bcolor = 'black' 
lwidth_border = 0.8
gcolor = 'xkcd:gunmetal'
linewidth_grid = 0.1

for i in np.arange(0,np.shape(ax)[0],1):
    ax[i].coastlines(resolution='50m', color=bcolor, linewidth=lwidth_border, zorder=2, alpha=1)
    gl = ax[i].gridlines(xlocs=np.arange(-180,180,30), ylocs=[0,30,60,90], linewidth=linewidth_grid, linestyle=[(0, (2.0, 2.0))], color=gcolor, zorder=6, alpha=0.3) #linestyles=[(0, (2.0, 2.0))], color=gcolor, zorder=6, alpha=0.3)
    ax[i].set_rasterization_zorder(1)

# Set up DT colormap range, norm:
bounds = np.arange(0,56,1)
norm = colors.BoundaryNorm(boundaries=bounds, ncolors=128, extend='max')

#######
# Specify plots:

# DT level - original package:
sp0 = ax[0].pcolormesh(file_old.longitude, file_old.latitude, file_old.iLev_DT, norm=norm, shading='gouraud', cmap=cc.white_rainbow(), transform=ccrs.PlateCarree(), zorder=0) 
ax[0].set_title(r'Dynamic Tropopause Model Level $-$ Original Diagnostics Package', fontsize=15.5, y=1.02, fontweight='medium', loc='center', horizontalalignment='center')

# DT level - revised package:
sp1 = ax[1].pcolormesh(file_old.longitude, file_old.latitude, file_new.iLev_DT, norm=norm, shading='gouraud', cmap=cc.white_rainbow(),transform=ccrs.PlateCarree(), zorder=0) 
ax[1].set_title(r'Dynamic Tropopause Model Level $-$ Revised Diagnostics Package', fontsize=15.5, y=1.02, fontweight='medium', loc='center', horizontalalignment='center')
ax[1].set_title(str(t_end.day).zfill(2)+' '+str(t_end.month_name())+' '+str(t_end.year)+'\n '+str(t_end)[-8:]+'Z', 
                     fontsize=12.5, y=.93, linespacing=1.5, fontweight='medium', loc='right', ha='right', color='gray', zorder=12)

# DT potential temperature - original package:
sp2 = ax[2].pcolormesh(file_old.longitude, file_old.latitude, file_old.theta_pv, vmin=260, vmax=460, shading='nearest', cmap=cc.ccc_rainbow_light(), transform=ccrs.PlateCarree(), zorder=0) 
ax[2].set_title(r'Dynamic Tropopause Potential Temperature',fontsize=15.5, y=1.02, fontweight='medium', loc='center', horizontalalignment='center')

# DT potential temperature - revised package:
sp3 = ax[3].pcolormesh(file_old.longitude, file_old.latitude, file_new.theta_pv, vmin=260, vmax=460, shading='nearest', cmap=cc.ccc_rainbow_light(), transform=ccrs.PlateCarree(), zorder=0) 
ax[3].set_title(r'Dynamic Tropopause Potential Temperature', fontsize=15.5, y=1.02, fontweight='medium', loc='center', horizontalalignment='center')

#######
# Plot colorbars:

fig.subplots_adjust(bottom=0.1, top=0.9, left=0.1, right=0.9, wspace=0.05, hspace=0.18)

# DT level colorbar:
cb = fig.colorbar(sp0, extend='max', orientation="horizontal", pad=0.05, fraction=0.035, shrink=0.8, aspect=45, ax=ax[0:2], ticks=np.arange(0,60,5))
cb.ax.tick_params(labelsize=14) 
cb.minorticks_off()

# DT theta colorbar:
cb2 = fig.colorbar(sp2, extend='both', orientation="horizontal", pad=0.05, fraction=0.035, shrink=0.8, aspect=45, ax=ax[2:], ticks=np.arange(260,480,20))
cb2.ax.tick_params(labelsize=14) 

figname = 'mpas_DTtheta_orig_revised_03.00.36Z.pdf'
fig.savefig(figname, bbox_inches="tight")
print('saved')

plt.show()